In [35]:
import pandas as pd
%matplotlib inline
from pathlib import Path
from zipfile import ZipFile
import xlrd
import openpyxl

In [30]:
file_list = [i for i in Path('./Задание/Задание/').iterdir()]
file_list

# raw_df = pd.read_excel(file_list[3])


[WindowsPath('Задание/Задание/json.json'),
 WindowsPath('Задание/Задание/txt.txt'),
 WindowsPath('Задание/Задание/Задание 1.xlsx'),
 WindowsPath('Задание/Задание/Задание 2 и 3.txt'),
 WindowsPath('Задание/Задание/Задание 4 (не обязательное).txt')]

In [51]:
dfs = pd.read_excel(
    file_list[2],
    engine='openpyxl',
    sheet_name=[1, 2]  # Replace 0 and 1 with the actual sheet indices you want to read
)

df_doctors = dfs[1]  # Assign the first DataFrame to df_doctors
df_report = dfs[2]   

In [155]:
(df_doctors.merge(
        df_doctors['ФИО'].str.extract(
        r'(?P<surname>[А-я]+)\s+(?P<name>[А-я]+)\s+(?P<patronymic>[А-я]+)')
        .applymap(lambda x: x[0].upper() + x[1:].lower() if isinstance(x, str) else ''),
    left_index=True,right_index=True)
    ['Состояние']
)

0         Болен\n
1         здоров 
2         здоров 
3         Болен\n
4           болен
           ...   
33472       болен
33473     здоров 
33474       Болен
33475     здоров 
33476      Здоров
Name: Состояние, Length: 33477, dtype: object

In [154]:
df_doctors['Состояние'].value_counts() # chel statuses
# def status_normalization():

(
    df_doctors['Состояние']
        .str.extract(r'([А-я]+)',expand=False)
        .str.lower()
        .value_counts()
        )



Состояние
болен     16809
здоров    16668
Name: count, dtype: int64

In [88]:
df_doctors.loc[[52,73,33400],:]

,ФИО,ДР,Дата Осмотра,Состояние
52,Константинова Снежана Александровна,1992-10-04 00:00:00,2020-10-12,Здоров
73,Светличная Зоя Юрьевна,1948-06-19 00:00:00,2020-10-12,Здоров
33400,Ермакова Рада Анатольевна,1967-09-22 00:00:00,2020-10-12,Здоров
